In [1]:
import os
from langchain_openai import OpenAI
from langchain.memory.buffer import ConversationBufferMemory
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.chains.llm import LLMChain
import streamlit as st
from dotenv import load_dotenv, find_dotenv

from constants import FEW_SHOT_EXAMPLE
import warnings
warnings.filterwarnings("ignore")

In [2]:
load_dotenv(find_dotenv())

# load the API Key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "")

In [6]:
# intialize the model
llm_model = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0.3,
    max_tokens=1500
)

# intialize the Memory
memory = ConversationBufferMemory()

# Intialize the Prompt example
example_travel_prompt = PromptTemplate(
    input_variables=["travel_query", "travel_response"],
    template="""
        Role: You are an intelligent Travel Agent Advisor whose job is to plan a trip based on the source and destination provided by the customer query.
        Instruction: First completely analyse the request step by step and then provide your response in accurate, 
                    precise and point wise manner so that a customer easily understands the mode of transportation, 
                    fare for each mode of transportation, flight number, cab number, buses number. 
                    Also, suggest the alternate route/mode of transportation if not present directly from source and destination. 
                    Make sure not to provide any incorrect or unfeasible information that is not useful to the customer. 
        Query: {travel_query}
        Response: {travel_response}
""",
)

In [8]:
print(example_travel_prompt.format(**FEW_SHOT_EXAMPLE[0]))


        Role: You are an intelligent Travel Agent Advisor whose job is to plan a trip based on the source and destination provided by the customer query.
        Instruction: First completely analyse the request step by step and then provide your response in accurate, 
                    precise and point wise manner so that a customer easily understands the mode of transportation, 
                    fare for each mode of transportation, flight number, cab number, buses number. 
                    Also, suggest the alternate route/mode of transportation if not present directly from source and destination. 
                    Make sure not to provide any incorrect or unfeasible information that is not useful to the customer. 
        Query: Can you plan a trip from Shahdara, Delhi to London?
        Response: 
        Since this is a international travel request, let's understand the process of travelling from one country to another.
        First we have to book a flight which mi

In [16]:
# Intialize few short template with examples
few_shot_prompt = FewShotPromptTemplate(
    examples=FEW_SHOT_EXAMPLE,
    example_prompt=example_travel_prompt,
    suffix="Query: {travel_query}",
    input_variables=["travel_query", "travel_response"]
)

In [17]:
# Created a LLM chain to run the model
travel_chain = LLMChain(
    llm=llm_model,
    memory=memory,
    prompt=few_shot_prompt
)

In [18]:
print(travel_chain.invoke({"travel_query":"Plan a trip from Hauz Khas, Delhi to Goa?"})["text"])


Response: 
Since this is a domestic travel request, let's understand the process of travelling from one city to another city. 
First we have to book the mode of transport which we need to travel from source to destination. It might be flight/train/cab/buses.

There are no direct flights from Hauz Khas, Delhi to Goa. 
IndiGo flight number 6E 744 will first fly from Delhi to Mumbai which has layover of 2hrs 40 mins and then from Mumbai to Goa.
The nearest airport from Hauz Khas is DEL, Delhi and nearest airport from Goa is GOI. The total cost will be Rs. 10000.

To reach DEL airport you can make use of Cabs which will cost around Rs. 500. You can also take the Delhi Metro from Hauz Khas 
station to New Delhi station, then change to the Airport Express Line to reach the airport. The total cost for the metro 
will be Rs. 60.

For Train, you can book 02413 Rajdhani Special which starts from NDLS, Delhi at 04:25 PM and reach to MAO, Goa at 11:00 AM the next day.
The journey has 8 stops and 

In [12]:
print(travel_chain.invoke({"travel_query":"Plan a trip from New York to bangalore?"})["text"])


Response: 
Since this is an international travel request, let's understand the process of travelling from one country to another.
First we have to book a flight which might have some layover in between and might be a direct flight.To book a flight you can 
use any of the online service provider. 

The layover options are: Emirates flight number EK-202, layover of 3hrs 25 mins at Dubai International Airport and then to destination Bengaluru Kempegowda International Airport.
Total cost will be Rs. 74201 per person.

The direct options are: Air India flight number AI-101 has total cost Rs. 107833, United Airlines UA 48 has total cost Rs. 112985

Now, to reach airport to pick the flight, we need to either travel via Cab/Subway/Train/Buses. The nearest Airport for New York is John F. Kennedy International Airport (JFK)
Now there are no direct Train which goes to New York's airport.
So we have left with either Cab/Subway/Buses.

The cab can be booked via Uber/Lyft/Private Taxis. The fare wi

In [19]:
print(travel_chain.invoke({"travel_query": "Plan a trip from Shivaji park, Delhi to Saket, Delhi?"})["text"])


Response: 
Since this is a domestic travel request, let's understand the process of travelling from one city to another city. 
First we have to book the mode of transport which we need to travel from source to destination. It might be flight/train/cab/buses.

There are no direct flights available from Shivaji Park to Saket.

For train, you can book the Delhi Metro from Rajiv Chowk station to Saket station. The fare is Rs. 30 and the journey will take approximately 25 minutes.

For cab, you can book a ride from Uber/Ola or any other private taxi service. The fare will be around Rs. 200 to Rs. 300 and the journey will take approximately 40 minutes.

For buses, you can take the DTC bus number 543A from Shivaji Park to Saket. The fare will be Rs. 25 and the journey will take approximately 1 hour.

If you prefer a more direct route, you can also take the Delhi Metro from Rajiv Chowk station to Hauz Khas station and then change to the Yellow Line to reach Saket station. This route will take

In [18]:
print(travel_chain.invoke({"travel_query":"Plan a trip from panchkula, harayana to Kashmir?"})["text"])



Since this is a domestic travel request, let's understand the process of travelling from one city to another city. 
First we have to book the mode of transport which we need to travel from source to destination. It might be flight/train/cab/buses.

For this request, the best mode of transportation would be a flight. The nearest airport to Panchkula, Haryana is Chandigarh Airport (IXC)
and the nearest airport to Kashmir is Srinagar Airport (SXR). There are no direct flights available from Chandigarh Airport to Srinagar Airport.
However, there are connecting flights with layovers.

The options for connecting flights are:

1. Air India flight AI 9836 from Chandigarh to Delhi, with a layover of 2 hours and then connecting flight AI 825 from Delhi to Srinagar.
Total cost would be around Rs. 11,000 to Rs. 15,000.

2. IndiGo flight 6E 251 from Chandigarh to Delhi, with a layover of 2 hours and then connecting flight 6E 555 from Delhi to Srinagar.
Total cost would be around Rs. 10,000 to Rs.

In [14]:
print(travel_chain.invoke({"travel_query": "Trip from Jamshedpur, Jharkhand to Rishikesh, Uttarkand?"})["text"])


Response: 
Since this is a domestic travel request, let's understand the process of travelling from one city to another city. 
First we have to book the mode of transport which we need to travel from source to destination. It might be flight/train/cab/buses.

There are no direct flights from Jamshedpur to Rishikesh. The nearest airport from Jamshedpur is Sonari Airport (IXW) and the nearest airport
from Rishikesh is Jolly Grant Airport (DED). The total cost for the flight will be around Rs. 8000 to Rs. 10000.

To reach Sonari Airport, you can take a cab which will cost around Rs. 500 to Rs. 700. You can also take a train from Tatanagar Junction 
to Ranchi Junction and then from Ranchi Junction to Sonari Airport. The total cost for the train will be between Rs. 500 to Rs. 1000.

For buses, you can take a bus from Jamshedpur to Ranchi and then from Ranchi to Rishikesh. The total cost for the bus will be between 
Rs. 1000 to Rs. 1500. The nearest bus station from Jamshedpur is Tatanagar 

In [15]:
print(travel_chain.invoke({"travel_query": "Trip from Manali to Vietnam?"})["text"])


Response: 
Since this is an international travel request, let's understand the process of travelling from one country to another.
First we have to book a flight which might have some layover in between and might be a direct flight. To book a flight you can 
use any of the online service provider. 

The layover options are: Air India flight number AI-981, layover of 4hrs 20 mins at Delhi and then to destination Hanoi, Vietnam.
Total cost will be Rs. 20800 per person.

The direct options are: Vietnam Airlines flight number VN-118 has total cost Rs. 47500, Vietjet Air VJ-181 has total cost Rs. 48200.

Now, to reach airport to pick the flight, we need to either travel via Cab/Bus. The nearest Airport for Manali is Bhuntar Airport.
From Manali, you can take a cab which will cost around Rs. 1500 to reach Bhuntar Airport.

For buses, you can take a bus from Manali to Delhi and then from Delhi to Hanoi, Vietnam. The total cost will be around Rs. 2000
and the journey will take approximately 2 